In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import torch
import numpy as np
from datetime import datetime
import os

In [2]:
def load_geomag_data(netcdf_path: str) -> xr.Dataset:
    """Load geomagnetic data from a NetCDF file."""
    try:
        print("loading raw data...")
        ds = xr.open_dataset(netcdf_path, chunks="auto")
        print(ds)
        return ds
    except Exception as e:
        raise RuntimeError(f"Error loading NetCDF file {netcdf_path}: {e}")

geomag_path = r"D:\earthquake-prediction\data\geomagnetic_data\supermag\all_stations_all2024.netcdf"
ds = load_geomag_data(geomag_path)

loading raw data...
<xarray.Dataset> Size: 7GB
Dimensions:  (block: 527040, vector: 201)
Dimensions without coordinates: block, vector
Data variables: (12/23)
    dbe_geo  (block, vector) float32 424MB dask.array<chunksize=(296940, 113), meta=np.ndarray>
    dbe_nez  (block, vector) float32 424MB dask.array<chunksize=(296940, 113), meta=np.ndarray>
    dbn_geo  (block, vector) float32 424MB dask.array<chunksize=(296940, 113), meta=np.ndarray>
    dbn_nez  (block, vector) float32 424MB dask.array<chunksize=(296940, 113), meta=np.ndarray>
    dbz_geo  (block, vector) float32 424MB dask.array<chunksize=(296940, 113), meta=np.ndarray>
    dbz_nez  (block, vector) float32 424MB dask.array<chunksize=(296940, 113), meta=np.ndarray>
    ...       ...
    time_dy  (block) int16 1MB dask.array<chunksize=(527040,), meta=np.ndarray>
    time_hr  (block) int16 1MB dask.array<chunksize=(527040,), meta=np.ndarray>
    time_mo  (block) int16 1MB dask.array<chunksize=(527040,), meta=np.ndarray>
    tim

In [3]:


def assign_time_as_coordinate(ds: xr.Dataset) -> xr.Dataset:
    time_as_datetime = pd.to_datetime({
        "year": ds["time_yr"].values,
        "month": ds["time_mo"].values,
        "day": ds["time_dy"].values,
        "hour": ds["time_hr"].values,
        "minute": ds["time_mt"].values
    })


    ds = ds.assign_coords(time=("block", time_as_datetime))
    ds = ds.drop_vars(["time_yr", "time_mo", "time_dy", "time_hr", "time_mt", "time_sc"])
    ds = ds.swap_dims({"block": "time"})
    
    return ds


ds = assign_time_as_coordinate(ds)
print(ds)
print(ds.indexes)

<xarray.Dataset> Size: 7GB
Dimensions:  (time: 527040, vector: 201)
Coordinates:
  * time     (time) datetime64[ns] 4MB 2024-01-01 ... 2024-12-31T23:59:00
Dimensions without coordinates: vector
Data variables: (12/17)
    dbe_geo  (time, vector) float32 424MB dask.array<chunksize=(296940, 113), meta=np.ndarray>
    dbe_nez  (time, vector) float32 424MB dask.array<chunksize=(296940, 113), meta=np.ndarray>
    dbn_geo  (time, vector) float32 424MB dask.array<chunksize=(296940, 113), meta=np.ndarray>
    dbn_nez  (time, vector) float32 424MB dask.array<chunksize=(296940, 113), meta=np.ndarray>
    dbz_geo  (time, vector) float32 424MB dask.array<chunksize=(296940, 113), meta=np.ndarray>
    dbz_nez  (time, vector) float32 424MB dask.array<chunksize=(296940, 113), meta=np.ndarray>
    ...       ...
    mcolat   (time, vector) float32 424MB dask.array<chunksize=(296940, 113), meta=np.ndarray>
    mlat     (time, vector) float32 424MB dask.array<chunksize=(296940, 113), meta=np.ndarray>
    

In [14]:

print(ds["id"].isel(time=0).values)

['AAA' 'ABG' 'ABK' 'AIA' 'AND' 'API' 'ARS' 'ASC' 'ASP' 'BDV' 'BEL' 'BFE'
 'BFO' 'BJN' 'BLC' 'BMT' 'BOU' 'BRD' 'BRW' 'BSL' 'C03' 'C04' 'C05' 'C06'
 'C09' 'C13' 'CBB' 'CKI' 'CLF' 'CMO' 'CNB' 'CPS' 'CSY' 'CTA' 'CYG' 'DAW'
 'DED' 'DLT' 'DMH' 'DOB' 'DON' 'DOU' 'DUR' 'EAG' 'EBR' 'ESK' 'EYR' 'FCC'
 'FMC' 'FRD' 'FRN' 'FSP' 'FUR' 'FYU' 'GAK' 'GAN' 'GCK' 'GDH' 'GIM' 'GLK'
 'GNG' 'GUA' 'GUI' 'HAD' 'HAN' 'HBK' 'HER' 'HLP' 'HOB' 'HON' 'HOP' 'HOV'
 'HRB' 'HRN' 'HUA' 'IPM' 'IQA' 'IRT' 'ISL' 'IVA' 'JAI' 'JAN' 'JCK' 'KAG'
 'KAK' 'KAR' 'KDU' 'KEP' 'KEV' 'KHB' 'KIL' 'KIR' 'KMH' 'KNY' 'KOU' 'KTB'
 'KUV' 'LER' 'LON' 'LRM' 'LRV' 'LVV' 'LYC' 'LYR' 'MAB' 'MAS' 'MAW' 'MCM'
 'MCQ' 'MEA' 'MEK' 'MGD' 'MMB' 'MSR' 'MUO' 'NAL' 'NAQ' 'NCK' 'NEW' 'NGK'
 'NOR' 'NR2' 'NUR' 'NVS' 'ORC' 'OTT' 'OUJ' 'PAF' 'PAG' 'PEG' 'PEL' 'PET'
 'PHU' 'PIL' 'PIN' 'PKR' 'PPT' 'PST' 'RAL' 'RAN' 'RES' 'RIK' 'ROE' 'RVK'
 'SBA' 'SBL' 'SHE' 'SHU' 'SIT' 'SJG' 'SKT' 'SMI' 'SOD' 'SOL' 'SOR' 'SPA'
 'SPT' 'STF' 'STJ' 'SUA' 'T03' 'T39' 'T43' 'T49' 'T

In [16]:
print(ds.dtype)

object


In [15]:
import cupy as cp

ds = ds.isel(vector=0)
ds = ds.to_array().unify_chunks()
ds = ds.map_blocks(cp.asarray)

Exception: Cannot infer object returned from running user provided function. Please supply the 'template' kwarg to map_blocks.

Exception: Cannot infer object returned from running user provided function. Please supply the 'template' kwarg to map_blocks.

In [20]:

print(ds[[ "dbn_nez", "dbe_nez", "dbz_nez", "glat", "glon"]].isel(block=1, vector=1).compute())

<xarray.Dataset> Size: 20B
Dimensions:  ()
Data variables:
    dbn_nez  float32 4B 4.452
    dbe_nez  float32 4B 1.046
    dbz_nez  float32 4B -0.7833
    glat     float32 4B 18.62
    glon     float32 4B 72.87
Attributes:
    description:  For the ground magnetometer data we gratefully acknowledge:...


In [9]:
print(ds.data_vars)

Data variables:
    dbe_geo  (block, vector) float32 424MB dask.array<chunksize=(296940, 113), meta=np.ndarray>
    dbe_nez  (block, vector) float32 424MB dask.array<chunksize=(296940, 113), meta=np.ndarray>
    dbn_geo  (block, vector) float32 424MB dask.array<chunksize=(296940, 113), meta=np.ndarray>
    dbn_nez  (block, vector) float32 424MB dask.array<chunksize=(296940, 113), meta=np.ndarray>
    dbz_geo  (block, vector) float32 424MB dask.array<chunksize=(296940, 113), meta=np.ndarray>
    dbz_nez  (block, vector) float32 424MB dask.array<chunksize=(296940, 113), meta=np.ndarray>
    decl     (block, vector) float32 424MB dask.array<chunksize=(296940, 113), meta=np.ndarray>
    extent   (block) float64 4MB dask.array<chunksize=(527040,), meta=np.ndarray>
    glat     (block, vector) float32 424MB dask.array<chunksize=(296940, 113), meta=np.ndarray>
    glon     (block, vector) float32 424MB dask.array<chunksize=(296940, 113), meta=np.ndarray>
    id       (block, vector) <U3 1GB d

In [30]:

start_time = {"time_yr": 2024, "time_mo": 1, "time_dy": 1, "time_hr": 0, "time_mt": 0}  # Start: Jan 1, 2025, 00:00
end_time = {"time_yr": 2024, "time_mo": 1, "time_dy": 7, "time_hr": 0, "time_mt": 0}    # End: Jan 2, 2025, 00:00

# Step 1: Select variables of interest
selected_vars = ds[["dbn_nez", "dbe_nez", "dbz_nez", "glat", "glon"]]


time_mask = (
    (ds["time_yr"] >= start_time["time_yr"]) & (ds["time_yr"] <= end_time["time_yr"]) &
    (ds["time_mo"] >= start_time["time_mo"]) & (ds["time_mo"] <= end_time["time_mo"]) &
    (ds["time_dy"] >= start_time["time_dy"]) & (ds["time_dy"] <= end_time["time_dy"]) 
    # (ds["time_hr"] >= start_time["time_hr"]) & (ds["time_hr"] <= end_time["time_hr"]) &
    # (ds["time_mt"] >= start_time["time_mt"]) & (ds["time_mt"] <= end_time["time_mt"])
).compute()

# Step 3: Apply the time mask
selected_data = selected_vars.where(time_mask, drop=True)
# Step 4: Compute the selection (load into memory)
final_data = selected_data.sel(vector=0).compute()

# Step 5: Print the result
print(final_data)

<xarray.Dataset> Size: 202kB
Dimensions:  (block: 10080)
Dimensions without coordinates: block
Data variables:
    dbn_nez  (block) float32 40kB 4.957 4.903 4.831 4.864 ... 8.531 8.304 8.188
    dbe_nez  (block) float32 40kB 2.935 2.933 2.732 ... -2.593 -2.682 -2.68
    dbz_nez  (block) float32 40kB -2.558 -2.558 -2.559 ... -3.162 -3.26 -3.263
    glat     (block) float32 40kB 43.25 43.25 43.25 43.25 ... 43.25 43.25 43.25
    glon     (block) float32 40kB 76.92 76.92 76.92 76.92 ... 76.92 76.92 76.92
Attributes:
    description:  For the ground magnetometer data we gratefully acknowledge:...


In [ ]:


def filter_data_by_time(ds: xr.Dataset, start_time: pd.Timestamp, end_time: pd.Timestamp) -> xr.DataArray:
    """
    Filters an xarray dataset based on a given time range.
    
    Parameters:
        ds (xr.Dataset): The dataset containing time-related variables.
        start_time (pd.Timestamp): The start time for filtering.
        end_time (pd.Timestamp): The end time for filtering.
    
    Returns:
        xr.DataArray: The filtered dataset with the specified time range applied.
    """
    # Step 1: Select variables of interest
    selected_vars = ds[["dbn_nez", "dbe_nez", "dbz_nez", "glat", "glon"]]

    # Step 2: Create a time mask based on the extracted time components
    time_mask = (
        (ds["time_yr"] >= start_time.year) & (ds["time_yr"] <= end_time.year) &
        (ds["time_mo"] >= start_time.month) & (ds["time_mo"] <= end_time.month) &
        (ds["time_dy"] >= start_time.day) & (ds["time_dy"] <= end_time.day)
        # (ds["time_hr"] >= start["time_hr"]) & (ds["time_hr"] <= end["time_hr"]) &
        # (ds["time_mt"] >= start["time_mt"]) & (ds["time_mt"] <= end["time_mt"])
    ).compute()

    # Step 3: Apply the time mask
    selected_data = selected_vars.where(time_mask, drop=True)

    # Step 4: Compute the selection (load into memory)
    final_data = selected_data.sel(vector=0).compute()

    return final_data

start_time = pd.Timestamp("2024-01-01 00:00:00")
end_time = pd.Timestamp("2024-01-07 00:00:00")

# Assuming `ds` is your xarray dataset
filtered_result = filter_data_by_time(ds, start_time, end_time)

# Print the result
print(filtered_result)


<xarray.Dataset> Size: 202kB
Dimensions:  (block: 10080)
Dimensions without coordinates: block
Data variables:
    dbn_nez  (block) float32 40kB 4.957 4.903 4.831 4.864 ... 8.531 8.304 8.188
    dbe_nez  (block) float32 40kB 2.935 2.933 2.732 ... -2.593 -2.682 -2.68
    dbz_nez  (block) float32 40kB -2.558 -2.558 -2.559 ... -3.162 -3.26 -3.263
    glat     (block) float32 40kB 43.25 43.25 43.25 43.25 ... 43.25 43.25 43.25
    glon     (block) float32 40kB 76.92 76.92 76.92 76.92 ... 76.92 76.92 76.92
Attributes:
    description:  For the ground magnetometer data we gratefully acknowledge:...


In [ ]:
'''
for each station k:

    #func 1 get positives
    eq_periods: List[Tuple[datetime, datetime]]
    for each eq in catalog:
        if eq in range:
            eq_periods.append((eq.time - timeperiod, eq.time))
    
            
    #func 2 get negatives, since eq_periods are sorted
    start_time = year/1/1 00:00:00
    end_time = start_time + timeperiod
    eq_index = 0
    normal_periods = List[Tuple[datetime, datetime]]
    while end_time < year+1/1/1 00:00:00:
        if end_time <= eq_periods[eq_index].first:
            period is normal, normal_periods.append((start_time, end_time))
            start_time = end_time
            end_time = start_time + timeperiod
        else:
            eq_index += 1
            start_time = eq_periods[eq_index].second
            end_time = start_time + timeperiod
    
    extract_geomag(station=k, eq_periods)
    extract_geomag(station=k, normal_periods)
            

'''

In [14]:
import xarray as xr
import pandas as pd
from haversine import haversine_vector, Unit
import numpy as np


In [39]:


def get_station_coords(geomag: xr.Dataset) -> np.ndarray:
    glats = geomag["glat"].isel(block=0).values
    glons = geomag["glon"].isel(block=0).values

    coords = np.stack((glats, glons), axis=-1)
    return coords


def normalize_coordinates_batch(coords: np.ndarray) -> np.ndarray:

    assert coords.shape[1] == 2, f"Invalid shape: {coords.shape}, expected (N, 2)"
    
    # Extract latitude and longitude
    lat, lon = coords[:, 0], coords[:, 1]

    # Validate latitude (raises error if out of range)
    if np.any((lat < -90) | (lat > 90)):
        raise ValueError("Latitude values must be between -90 and 90.")

    # Normalize longitude to range [-180, 180] using vectorized modulo operation
    lon = ((lon + 180) % 360) - 180

    # Return new array with normalized values
    return np.column_stack((lat, lon))


def get_eq_periods_for_station(
        station_coord: np.ndarray, 
        catalog: pd.DataFrame, 
        time_period: pd.Timedelta, 
        radius: float) -> np.ndarray:
    
    station_coord = normalize_coordinates_batch(station_coord.reshape(1, 2))[0]

    coords = np.stack((catalog["latitude"].values, catalog["longitude"].values), axis=1)
    # print(coords)
    coords = normalize_coordinates_batch(coords)
    
    distances = haversine_vector(coords, station_coord, unit=Unit.KILOMETERS, comb=True )[0]  # Fast haversine distance
    # print(distances)
    mask = distances <= radius
    # print(mask)
    filtered_eqs = catalog.loc[mask]
    filtered_eqs = pd.to_datetime(filtered_eqs["time"], format="%Y-%m-%dT%H:%M:%S.%fZ", utc=True)

    # Compute time windows efficiently
    eq_periods = np.column_stack([
        (filtered_eqs - time_period).values,  # Start times
        filtered_eqs.values  # End times
    ])

    return eq_periods


def get_normal_periods_for_station(
        year: int,
        eq_periods: list[tuple[pd.Timestamp, pd.Timestamp]],
        time_period: pd.Timedelta,
        ) -> np.ndarray:
    
    start_time = pd.Timestamp(year=year, month=1, day=1, hour=0, minute=0, second=0)
    end_of_year = pd.Timestamp(year=year+1, month=1, day=1, hour=0, minute=0, second=0)

    normal_periods = []
    
    eq_index = 0
    end_time = start_time + time_period

    while end_time < end_of_year:
        if eq_index < len(eq_periods) and end_time <= eq_periods[eq_index][0]:  
            # No overlap, normal period
            normal_periods.append([start_time, end_time])
            start_time = end_time
            end_time = start_time + time_period
        elif eq_index < len(eq_periods):
            # Overlap detected, move start_time to after the earthquake period
            start_time = eq_periods[eq_index][1]
            end_time = start_time + time_period
            eq_index += 1  # Move to the next earthquake period
        else:
            # No more earthquakes, fill in remaining normal periods
            normal_periods.append([start_time, end_time])
            start_time = end_time
            end_time = start_time + time_period  

    return np.array(normal_periods, dtype=object)



In [34]:
catalog = pd.read_csv(r"D:\earthquake-prediction\data\catalog\csv\usgs\2024.csv")
# needs function that extracts the coordinates for each station
station_coords = get_station_coords(ds)
time_period = pd.Timedelta(days=7)
radius = 200
year = 2024

dataset_ranges_by_station = np.empty((0, 2), dtype=object)
# iterate through each station
for station_coord in station_coords:
    
    # get earthquake periods for station
    eq_periods = get_eq_periods_for_station(
        station_coord=station_coord, 
        catalog=catalog, 
        time_period=time_period, 
        radius=radius)
    # get normal periods for station
    normal_periods = get_normal_periods_for_station(
        eq_periods=eq_periods,
        time_period=time_period, 
        year=year)
    
    print(f"Station: {station_coord}")
    print(f"Earthquake periods: {eq_periods.shape}")
    print(f"Normal periods: {normal_periods.shape}\n")

    station_samples = np.array([normal_periods, eq_periods], dtype=object)
    print(station_samples.shape)
    dataset_ranges_by_station = np.vstack([dataset_ranges_by_station, station_samples])

print(f"Dataset shape: {dataset_ranges_by_station.shape}")




[16058.26460867  9215.43814982  9092.46640075 ...  5021.83967847
  5041.58618265  5039.5674001 ]
[False False False ... False False False]
Station: [43.25 76.92]
Earthquake periods: (0, 2)
Normal periods: (52, 2)

(2,)
[16189.86430991  8962.30748442  8973.32457885 ...  6490.33910363
  6529.43764217  6527.50236207]
[False False False ... False False False]
Station: [18.62 72.87]
Earthquake periods: (0, 2)
Normal periods: (52, 2)

(2,)
[12064.69202844 12298.95748935 12054.2917633  ...  7219.38979841
  7202.56355618  7201.2582046 ]
[False False False ... False False False]
Station: [68.35 18.82]
Earthquake periods: (0, 2)
Normal periods: (52, 2)

(2,)
[ 5032.48540266 11525.03011201 11777.43038535 ... 16605.24032726
 16625.11269278 16626.32299026]
[False False False ... False False False]
Station: [-65.245 295.742]
Earthquake periods: (0, 2)
Normal periods: (52, 2)

(2,)
[11971.28540296 12318.97651861 12069.38558594 ...  7237.24982177
  7219.14330125  7217.88003849]
[False False False ... 

In [42]:
catalog = pd.read_csv(r"D:\earthquake-prediction\data\catalog\csv\usgs\2024.csv")
# needs function that extracts the coordinates for each station
station_coords = get_station_coords(ds)
time_period = pd.Timedelta(days=7)
radius = 200
year = 2024

dataset_ranges_by_station = {}
# iterate through each station
id = 0
total_pos = 0
total_neg = 0
for station_coord in station_coords:
    
    # get earthquake periods for station
    eq_periods = get_eq_periods_for_station(
        station_coord=station_coord, 
        catalog=catalog, 
        time_period=time_period, 
        radius=radius)
    # get normal periods for station
    normal_periods = get_normal_periods_for_station(
        eq_periods=eq_periods,
        time_period=time_period, 
        year=year)
    
    # print(f"Station: {station_coord}")
    total_pos += eq_periods.shape[0]
    total_neg += normal_periods.shape[0]

    station_dict = {}
    station_dict["negative"] = normal_periods
    station_dict["positive"] = eq_periods

    # print(station_dict)

    dataset_ranges_by_station[id] = station_dict
    id += 1

print(f"Dataset : {len(dataset_ranges_by_station)}")
print(f"total positive samples: {total_pos}")
print(f"total negative samples: {total_neg}")




Dataset : 201
total positive samples: 104
total negative samples: 10738


In [47]:
time_as_datetime = pd.to_datetime({
    "year": ds["time_yr"].values,
    "month": ds["time_mo"].values,
    "day": ds["time_dy"].values,
    "hour": ds["time_hr"].values,
    "minute": ds["time_mt"].values
})

print(time_as_datetime.shape)
print(time_as_datetime[2])

(527040,)
2024-01-01 00:02:00


In [49]:
def assign_time_as_coordinate(ds: xr.Dataset) -> xr.Dataset:
    time_as_datetime = pd.to_datetime({
        "year": ds["time_yr"].values,
        "month": ds["time_mo"].values,
        "day": ds["time_dy"].values,
        "hour": ds["time_hr"].values,
        "minute": ds["time_mt"].values
    })

    ds = ds.assign_coords(time=("block", time_as_datetime))
    ds = ds.drop_vars(["time_yr", "time_mo", "time_dy", "time_hr", "time_mt"])
    ds = ds.swap_dims({"block": "time"})

    print(ds)

    
    return ds

ds2 = assign_time_as_coordinate(ds)

<xarray.Dataset> Size: 7GB
Dimensions:  (time: 527040, vector: 201)
Coordinates:
  * time     (time) datetime64[ns] 4MB 2024-01-01 ... 2024-12-31T23:59:00
Dimensions without coordinates: vector
Data variables: (12/18)
    dbe_geo  (time, vector) float32 424MB dask.array<chunksize=(296940, 113), meta=np.ndarray>
    dbe_nez  (time, vector) float32 424MB dask.array<chunksize=(296940, 113), meta=np.ndarray>
    dbn_geo  (time, vector) float32 424MB dask.array<chunksize=(296940, 113), meta=np.ndarray>
    dbn_nez  (time, vector) float32 424MB dask.array<chunksize=(296940, 113), meta=np.ndarray>
    dbz_geo  (time, vector) float32 424MB dask.array<chunksize=(296940, 113), meta=np.ndarray>
    dbz_nez  (time, vector) float32 424MB dask.array<chunksize=(296940, 113), meta=np.ndarray>
    ...       ...
    mlat     (time, vector) float32 424MB dask.array<chunksize=(296940, 113), meta=np.ndarray>
    mlon     (time, vector) float32 424MB dask.array<chunksize=(296940, 113), meta=np.ndarray>
    

In [64]:


def extract_multiple_segments(
        sample_ranges: np.ndarray,
        geomag: xr.Dataset,
        station_idx: int,
        label: int) -> xr.Dataset:

    geomag = geomag.isel(vector=station_idx)
    segments = [geomag.sel(time=slice(start, end)).expand_dims(sample=[f"station: {station_idx}, label: {label}, idx: {i}"]) 
                for i, (start, end) in enumerate(sample_ranges)]
    
    # Concatenate along the new "sample" dimension
    ds = xr.concat(segments, dim="sample")

    if ds is not None:
        # ds= ds.assign_coords(sample=("sample", np.arange(len(sample_ranges))))
        ds = ds.assign({"labels": ("sample", np.full(len(sample_ranges), label, dtype=int))})

    return ds

ranges = np.array([
    [pd.Timestamp("2024-01-01 02:00"), pd.Timestamp("2024-01-01 04:00")],  # First segment
    [pd.Timestamp("2024-01-01 07:00"), pd.Timestamp("2024-01-01 09:00")]   # Second segment
])


# Extract separate time segments
pos_ds = extract_multiple_segments(geomag=ds2, sample_ranges=ranges, label= 1, station_idx=0)
neg_ds = extract_multiple_segments(geomag=ds2, sample_ranges=ranges, label= 0, station_idx=0)
if pos_ds is not None and neg_ds is not None:
    combined_ds = xr.concat([pos_ds, neg_ds], dim="sample")
elif pos_ds is not None:
    combined_ds = pos_ds
elif neg_ds is not None:
    combined_ds = neg_ds
else:
    raise ValueError("No valid time ranges provided.")

print(combined_ds)

<xarray.Dataset> Size: 87kB
Dimensions:  (sample: 4, time: 242)
Coordinates:
  * time     (time) datetime64[ns] 2kB 2024-01-01T02:00:00 ... 2024-01-01T09:...
  * sample   (sample) object 32B 'station: 0, label: 1, idx: 0' ... 'station:...
Data variables: (12/19)
    dbe_geo  (sample, time) float32 4kB dask.array<chunksize=(1, 121), meta=np.ndarray>
    dbe_nez  (sample, time) float32 4kB dask.array<chunksize=(1, 121), meta=np.ndarray>
    dbn_geo  (sample, time) float32 4kB dask.array<chunksize=(1, 121), meta=np.ndarray>
    dbn_nez  (sample, time) float32 4kB dask.array<chunksize=(1, 121), meta=np.ndarray>
    dbz_geo  (sample, time) float32 4kB dask.array<chunksize=(1, 121), meta=np.ndarray>
    dbz_nez  (sample, time) float32 4kB dask.array<chunksize=(1, 121), meta=np.ndarray>
    ...       ...
    mlon     (sample, time) float32 4kB dask.array<chunksize=(1, 121), meta=np.ndarray>
    mlt      (sample, time) float32 4kB dask.array<chunksize=(1, 121), meta=np.ndarray>
    npnt     (s